In [1]:
import modules.text_processing
import torch
import torch.nn.functional as F
from imp import reload
from modules.text_processing import read_split_file, BatchWrapper, generate_iterators


In [2]:
#Read and split data
train_path, val_path, test_path = read_split_file("Data/poems_fixed.csv","tmp/")

In [2]:
#Create iterators
batch_size = 5
train_iter, val_iter, test_iter, TEXT = generate_iterators("split_used_for_baseline/poems_train.csv", "split_used_for_baseline/poems_val.csv", "split_used_for_baseline/poems_test.csv", 
                                                           batch_size = batch_size, device="cuda")



In [3]:
#Taken and modified from lab/hw
def evaluate(model, data_iter, batch_size):
    model.eval()
    val_loss = 0
    total = 0
    total_acc = 0
    val_acc = 0
    for i in range(len(data_iter)):
        vectors,labels = next(data_iter.__iter__())
        output = model(vectors)
        _, predicted = torch.topk(output.data, k=2, dim=1)
        predictions = torch.zeros(labels.size()).to("cuda")
        predictions.scatter_(1, predicted, 1)
        val_acc +=  ((predictions == labels) & (labels == 1)).sum().item()
        total_acc += 2*labels.size(0)
        val_loss += F.kl_div(output.log(), labels)
        total +=1
    return val_loss / total, val_acc / total_acc

In [4]:
#Taken and modified from lab/hw
def training_loop(batch_size, num_epochs, model, loss_, optim, training_iter, dev_iter,
                  train_eval_iter, test_iter, verbose=True):
    
    epoch = 0
    total_batches = int(len(training_iter))
    dev_accuracies = []
    test_accuracies = []
    while epoch <= num_epochs:
        print("Training...")
        for i in range(total_batches):
            model.train()
            vectors, labels = next(training_iter.__iter__())
            model.zero_grad()

            output = model(vectors)
            lossy = loss_(output.log(), labels)
            lossy.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optim.step()


        model.eval()
        print("Evaluating training....")
        #train_loss, train_acc = evaluate(model, train_eval_iter, batch_size)
        train_loss, train_acc = 0,0
        print("Evaluating dev...")
        eval_loss, eval_acc = evaluate(model, dev_iter, batch_size)
        test_loss, test_acc = evaluate(model, test_iter, batch_size)
        dev_accuracies.append(eval_acc)
        test_accuracies.append(test_acc)
        if verbose:
            print("Epoch %i; Loss %f; Train loss: %f; Dev loss %f, Dev acc: %f, Test acc: %f"  %(epoch, lossy.item(), train_acc, eval_loss, eval_acc, test_acc))
        epoch += 1    
    best_dev = max(dev_accuracies)
    best_test = test_accuracies[np.argmax(dev_accuracies)]
    return best_dev, best_test

In [5]:
import numpy as np
import pandas as pd
import modules.models
reload(modules.models)
from modules.models import BiLSTMBaseline

dropout_min = 0.2
dropout_max = 0.8

learning_rate_min = 0.0001
learning_rate_max = 0.01

hidden_dim_min = 50
hidden_dim_max = 300

v_dim_min = 5
v_dim_max = 30
emotions = ['anger','anticipation','fear','joy','love','optimism','pess','sad']
label_size = 8
embedding_dim = 300

dropouts = []
learning_rates = []
hidden_dims = []
v_dims = []
dev_accuracies = []
test_accuracies = []

for i in range(60):
    print("Iteration: ", i)
    #Create wrappers for iterators
    train_batch = BatchWrapper(train_iter, "text", emotions)
    train_eval_batch = BatchWrapper(train_iter, "text", emotions)
    valid_batch = BatchWrapper(val_iter, "text", emotions)
    test_batch = BatchWrapper(test_iter, "text", emotions)
    
    dropout = np.random.uniform(dropout_min, dropout_max)
    learning_rate = 10**np.random.uniform(np.log10(learning_rate_min), np.log10(learning_rate_max))
    hidden_dim = np.random.randint(hidden_dim_min, hidden_dim_max)
    v_dim = np.random.randint(v_dim_min, v_dim_max)
    
    m = BiLSTMBaseline(vocab_size = len(TEXT.vocab), embedding_dim = embedding_dim, hidden_dim = hidden_dim, label_size=label_size, 
                       v_dim = v_dim, pretrained_vec=TEXT.vocab.vectors, use_gpu = True, dropout = dropout)
    m.to("cuda")
    opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), learning_rate)
    dev_acc, test_acc = training_loop(model=m, training_iter=train_batch, dev_iter=valid_batch, train_eval_iter = train_eval_batch, test_iter = test_batch, loss_=F.kl_div, optim=opt, num_epochs=10, batch_size = batch_size)

    dropouts.append(dropout)
    learning_rates.append(learning_rate)
    hidden_dims.append(hidden_dim)
    v_dims.append(v_dim)
    dev_accuracies.append(dev_acc)
    test_accuracies.append(test_acc)
    df = pd.DataFrame({"droprate": dropouts, "learning_rates": learning_rates, "hidden_dim": hidden_dims, "v_dim": v_dims, "dev_accuracy": dev_accuracies, "test_accuracy": test_accuracies})
    df.to_csv("right.csv")

    
df = pd.DataFrame({"droprate": dropouts, "learning_rates": learning_rates, "hidden_dim": hidden_dims, "v_dim": v_dims, "dev_accuracy": dev_accuracies, "test_accuracy": test_accuracies})

    


Iteration:  0
Training...


/home/philip/.local/lib/python3.7/site-packages/torch/nn/functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


Evaluating training....
Evaluating dev...
Epoch 0; Loss 0.497598; Train loss: 0.000000; Dev loss 0.512192, Dev acc: 0.339286, Test acc: 0.390071
Training...
Evaluating training....
Evaluating dev...
Epoch 1; Loss 0.498877; Train loss: 0.000000; Dev loss 0.507473, Dev acc: 0.375000, Test acc: 0.407801
Training...
Evaluating training....
Evaluating dev...
Epoch 2; Loss 0.478385; Train loss: 0.000000; Dev loss 0.520326, Dev acc: 0.330357, Test acc: 0.411348
Training...
Evaluating training....
Evaluating dev...
Epoch 3; Loss 0.517569; Train loss: 0.000000; Dev loss 0.506991, Dev acc: 0.392857, Test acc: 0.436170
Training...
Evaluating training....
Evaluating dev...
Epoch 4; Loss 0.478450; Train loss: 0.000000; Dev loss 0.513030, Dev acc: 0.343750, Test acc: 0.407801
Training...
Evaluating training....
Evaluating dev...
Epoch 5; Loss 0.452659; Train loss: 0.000000; Dev loss 0.491152, Dev acc: 0.424107, Test acc: 0.414894
Training...
Evaluating training....
Evaluating dev...
Epoch 6; Loss 0.

In [8]:
df.sort_values("dev_accuracy")

,droprate,learning_rates,hidden_dim,v_dim,dev_accuracy,test_accuracy
55,0.216915,0.006588,97,23,0.303571,0.336879
52,0.637862,0.003939,240,15,0.325893,0.280142
51,0.494909,0.001578,149,28,0.334821,0.336879
56,0.350545,0.000482,264,29,0.339286,0.297872
35,0.638593,0.004840,166,25,0.343750,0.280142
41,0.531746,0.004108,281,19,0.348214,0.368794
36,0.356340,0.006259,231,17,0.348214,0.354610
17,0.532029,0.003339,129,27,0.352679,0.322695
31,0.579366,0.004421,152,6,0.352679,0.368794
6,0.498583,0.002622,211,18,0.357143,0.297872
